# Test place for everything

# import functions

In [17]:
from sqlays import export_sql, import_sql
from matheq import Cel_to_Kel, Kel_to_Cel, to_pot_temp
from mapscre import station_map, bathy_data, contour_ver, TS_diagram
from iscays import sum_up, vertical_plot, depth_bin_interval, particle_bin_interval, isc_xlsx 
from ctdays import find_start_row, raw_ctd_to_df
from graphcre import deployment_constancy, vertical_each_org_distribution, stacked_vertical_distribution, comp_vertical_distribution 

# create TS diagram

In [ ]:
ctd_df = export_sql('ctd', 'ctd_meta')
#station_list = ['PS107_18']
#ctd_df = ctd_df.loc[ctd_df['Event'].str.contains('|'.join(station_list))]
ctd_df = ctd_df.loc[pd.to_numeric(ctd_df['Depth water [m]']) <=1000]
depth = tuple(pd.to_numeric(ctd_df['Depth water [m]']))
sal = tuple(pd.to_numeric(ctd_df['Sal']))
temp = tuple(pd.to_numeric(ctd_df['Temp [°C]']))
TSD_dict = {'temp': temp, 'sal': sal, 'depth':depth}
TS_diagram(TSD_dict)


# create contour map

In [ ]:
ctd_df = export_sql('ctd', 'ctd_meta'); print(list(ctd_df))
station_list = ['PS107_10', 'PS107_12', 'PS107_14', 'PS107_16', 'PS107_18']
cols_to_use = ['Latitude', 'Longitude', 'Depth water [m]', 'Press [dbar]']
ctd_df_filter = ctd_df.loc[ctd_df['Event'].str.contains('|'.join(station_list))] # select by station
#ctd_df_filter = ctd_df_filter[cols_to_use] # select only necessary columns
ctd_df_filter = ctd_df.loc[(pd.to_numeric(ctd_df['Latitude']) >=78.9) & (pd.to_numeric(ctd_df['Latitude']) <=79.1)] # select by lat and lon
ctd_df_filter = ctd_df_filter.loc[(pd.to_numeric(ctd_df_filter['Depth water [m]']) <=1000)]
ctd_df_filter = ctd_df_filter.dropna() # drop rows having empty value (empty values will affect contouring method)

ctd_df_filter['d_n'] = ctd_df_filter.apply(lambda row: day_night(float(row['Latitude']), float(row['Longitude']), datetime.strptime(row['Date/Time'], '%Y-%m-%dT%H:%M')), axis=1 )
ctd_df_filter = ctd_df_filter.loc[ctd_df_filter['d_n'] == 'day']

depth = [i*(-1) for i in list(pd.to_numeric(ctd_df_filter['Depth water [m]']).values)]
lat = list(pd.to_numeric(ctd_df_filter['Latitude']).values)
lon = list(pd.to_numeric(ctd_df_filter['Longitude']).values)
z = list(pd.to_numeric(ctd_df_filter['Press [dbar]']).values)

ctd_array = np.array((lat, lon, z, depth)).transpose() # create array

topo = bathy_data (75, 83, -30, 20)
contour_ver(topo, 'lat', 79, (-5, 10), ctd_array)
#contour_ver(topo, 'lon', 79, (-5, 10), ctd_array)





# create station map

In [ ]:
isc_df = export_sql('isc', 'isc_meta')
cruise_list = ('PS107', 'PS114', 'PS121')
cruise_dict = {}
for cruise in cruise_list:
    cruise_df = isc_df[isc_df['cruise_station_haul'].str.contains(cruise)]
    cruise_dict[cruise] = (tuple(pd.to_numeric(cruise_df['lat'])) , tuple(pd.to_numeric(cruise_df['lon'])))

topo = bathy_data (75, 83, -30, 20) # -30, 30, 75, 85
station_map(cruise_dict, topo)